# STATUS: Unstarted

In [ ]:
import json
import operator

In [ ]:
pool_depth = 100

In [ ]:
id_pos = 1
score_pos = 2
ans_length = 3 
id_length = 64

clus_type = 'Cluster Facet'

In [ ]:
def AvgPrecision(gt, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    rel_docs = len(gt)
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/rel_docs

In [ ]:
def clean_sub(sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    for entry in sub:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        if entry[index_of_ad] not in seen:
            uniqs.append(entry)
            seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

# STATUS: MUST UPDATE CORRECT ANSWERS

In [ ]:
round_one_dd_clustering_file =  '../../../data/annotation_prep/dd_clustering/FIRST_ROUND_seeds2cluster_ads.json'
round_two_dd_clustering_file =  '../../../data/annotation_prep/dd_clustering/SECOND_ROUND_seeds2cluster_ads.json'
#dd_clustering_file =  '../../../data/annotation_prep/dd_clustering/FINAL_seeds2cluster_ads.json'

print "MAKE SURE YOU UPDATED TO RIGHT FILE"

questions_file =  '../../../data/annotation_prep/dd_clustering/FIRST_ROUND_chosen_questions.json'

with open(round_one_dd_clustering_file, 'r') as f:
    round_one_seeds2cluster_ads = eval(f.read()) 
    
with open(round_two_dd_clustering_file, 'r') as f:
    round_two_seeds2cluster_ads = eval(f.read())   
    
with open(questions_file, 'r') as f:
    questions = eval(f.read())
    
seeds2cluster_ads = {}
for seed in round_one_seeds2cluster_ads.keys():
    temp = round_one_seeds2cluster_ads[seed]
    temp.extend(round_two_seeds2cluster_ads[seed])
    temp = list(set(temp))
    seeds2cluster_ads[seed] = temp

In [ ]:
question_ids = questions['NYU'][clus_type].keys()

### To avoid confusing dependecies, we'll get submission data from raw submissions

In [ ]:
submission_path = '../../../data/team_submissions/'

In [ ]:
answers = {}

# NYU

In [ ]:
answers['NYU'] = {}
answers['NYU'][clus_type] = {}

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_NYU_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
data = []
for entry in all_data:
    if entry['questionType'] == clus_type:
        data.append(entry)

for qid in question_ids:
    submissions = []
    for entry in data:
        # We are only evaluating some of them
        if entry['question_id'] == qid:
            submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['NYU'][clus_type][qid] = submissions

# HG

In [ ]:
answers['HG'] = {}
answers['HG'][clus_type] = {}

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_HG_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
data = []
for entry in all_data:
    if entry['questionType'] == clus_type:
        data.append(entry)

for qid in question_ids:
    submissions = []
    for entry in data:
        # We are only evaluating some of them
        if entry['question_id'] == qid:
            submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['HG'][clus_type][qid] = submissions

# JPL

In [ ]:
answers['JPL'] = {}
answers['JPL'][clus_type] = {}

In [ ]:
file_path = submission_path + ('Uncharted/DomainDiscovery/uncharted_JPL_DD.json')
f = open(file_path, 'r')
all_data = eval(f.read())
data = []
for entry in all_data:
    if entry['questionType'] == clus_type:
        data.append(entry)

for qid in question_ids:
    submissions = []
    for entry in data:
        # We are only evaluating some of them
        if entry['question_id'] == qid:
            if 'answers' in entry.keys():
                submissions = clean_sub(entry['answers'], id_pos, score_pos, ans_length)
            else:
                print entry['question_id']
                print 'Question 26 known to have no submitted answers'
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['JPL'][clus_type][qid] = submissions

# Calculate Results

In [ ]:
results = {}
results[clus_type] = {}

for dataset in answers.keys():
    results[clus_type][dataset] = {}
    for quest in answers[dataset][clus_type].keys():
        results[clus_type][dataset][quest] = {}
        submission = answers[dataset][clus_type][quest]
        seed = questions[dataset][clus_type][quest]['seed']
        relevant_ads = seeds2cluster_ads[seed]
        results[clus_type][dataset][quest] = {}
        if len(relevant_ads) != 0:
            results[clus_type][dataset][quest]['avg_precision'] = AvgPrecision(relevant_ads, submission, id_pos, score_pos)
        else:
            results[clus_type][dataset][quest]['avg_precision'] = 'NA'
            print seed

In [ ]:
output_file = 'Uncharted_DD_Cluster_Facet_Results_REWRITE.json'
with open(output_file, 'w') as outfile:
    json.dump(results, outfile, indent=2)

# DEBUG